In [67]:
import pandas as pd
import numpy as np
df=pd.read_csv("final_movie_data.csv")

In [68]:
df["tags"]=df["tags"].apply(lambda x:x.lower())

#tags vectorisation

In [69]:
#preprocessing tags
import nltk 
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet

def get_wordnet_pos(tag):
    if tag.startswith("J"):
        return wordnet.ADJ
    elif tag.startswith("V"):
        return wordnet.VERB
    elif tag.startswith("N"):
        return wordnet.NOUN
    elif tag.startswith("R"):
        return wordnet.ADV
    else:
        return wordnet.NOUN
lemmatizer=WordNetLemmatizer()
def lemmatize_tags(text):
    text_words=text.split()
    pos_tags=pos_tag(text_words)
    new_text=[]
    for word,tag in pos_tags:
        wordnet_pos=get_wordnet_pos(tag)
        lemmatized_word=lemmatizer.lemmatize(word,wordnet_pos)
        new_text.append(lemmatized_word)
    return " ".join(new_text)



In [70]:
df["tags"]=df["tags"].apply(lemmatize_tags)
import re
df['tags'] = df['tags'].apply(lambda x: re.sub(r'[^a-zA-Z\s]', ' ', x))


In [71]:
df["tags"][44]

'deckard shaw seek revenge against dominic toretto and his family for his comatose brother  action carrace speed revenge suspense car race musclecar vindiesel paulwalker dwaynejohnson michellerodriguez tyresegibson jameswan'

In [72]:
# count vectorisation
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words='english')
vectors=cv.fit_transform(df['tags']).toarray()
vectors.shape

(4806, 5000)

In [73]:
df.head(1)

,id,popularity,revenue,title,vote_average,vote_count,tags
0,19995,150.437577,2787965087,Avatar,7.2,11800,in the nd century a paraplegic marine be di...


In [74]:
len(cv.get_feature_names_out())

5000

In [75]:
from sklearn.metrics.pairwise import cosine_similarity

similarity=cosine_similarity(vectors)

In [76]:
similarity[0].shape

(4806,)

In [81]:
def recommend(movie):
    movie=movie.lower()
    if(movie not in df["title"].str.lower().values):
        print("Movie not found. Please check the spelling or try another movie.")
        return
    movie_index=df[df["title"].str.lower()==movie].index[0]
    distance=similarity[movie_index]
    movie_list=sorted(list(enumerate(distance)),reverse=True,key=lambda x:x[1])[1:6]
    
    for i in movie_list:
        print(df.iloc[i[0]].title)

In [83]:
recommend("iron man")


Iron Man 2
Iron Man 3
Avengers: Age of Ultron
Captain America: Civil War
The Avengers
